In [1]:
import sys
sys.path.append('../')

from src.finance_ml.data_preparation.data_preparation import DataLoader
from src.finance_ml.trading_strategy.pivot_strategy import *
from src.finance_ml.indicators.indicators import Indicators

## Load Data

In [2]:
file_time = '1h'
dataloader = DataLoader(time_index_col= 'DATE', 
                    keep_cols = ['VOLUME','OPEN', 'CLOSE', 'LOW', 'TRANSACTIONS','HIGHT'])
ticker_name = 'BTCUSD'
data = dataloader.load_dataset({ticker_name:'../data/cryptos/BTCUSD_2020-04-07_2022-04-06.parquet'})


## Resample OHLC Data into different timeframe

In [3]:
open_col = f'{ticker_name}_OPEN'
high_col = f'{ticker_name}_HIGHT'
low_col = f'{ticker_name}_LOW'
close_col = f'{ticker_name}_CLOSE'

resampled_data_5min = resample_ohlc(data,[open_col, high_col, low_col, close_col],'5min')
resampled_data_15min = resample_ohlc(data,[open_col, high_col, low_col, close_col],'15min')
resampled_data_30min = resample_ohlc(data,[open_col, high_col, low_col, close_col],'30min')
resampled_data_1hr = resample_ohlc(data,[open_col, high_col, low_col, close_col],'1h')
resampled_data_1d = resample_ohlc(data,[open_col, high_col, low_col, close_col],'1d')
resampled_data_1wk = resample_ohlc(data,[open_col, high_col, low_col, close_col],'1W')

D:\FAU\Ml in finance_ex\github_repo\notebooks\..\src\finance_ml\trading_strategy\pivot_strategy.py:85: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  resampled_data.index = resampled_data.index + pd.DateOffset(weekday=0)


## Calculating Pivot Points

In [4]:
woodie_pivot_1d = woodie_pivot(resampled_data_1d,high_col,low_col,close_col)
woodie_pivot_1wk = woodie_pivot(resampled_data_1wk,high_col,low_col,close_col)

In [5]:
woodie_pivot_1d

,DATE,Pivot,S1,S2,S3,R1,R2,R3
1,2020-04-08,7325.911228,7047.930000,6911.265000,6749.530000,7346.330000,7508.065000,7644.730000
2,2020-04-09,7264.550000,7221.822456,7047.556140,6943.467368,7500.177544,7604.266316,7778.532632
3,2020-04-10,6948.672500,7151.000000,6978.350000,6864.800000,7437.200000,7550.750000,7723.400000
4,2020-04-11,6874.980000,6591.245000,6383.562500,6026.135000,7156.355000,7513.782500,7721.465000
5,2020-04-12,6958.122500,6789.960000,6675.700000,6590.680000,6989.240000,7074.260000,7188.520000
...,...,...,...,...,...,...,...,...
725,2022-04-02,46132.135000,45042.417375,43383.716062,42534.572125,47550.262625,48399.406563,50058.107875
726,2022-04-03,46487.330000,45024.270000,44528.135000,43420.270000,46628.270000,47736.135000,48232.270000
727,2022-04-04,46301.165000,45374.660000,44392.330000,43279.660000,47469.660000,48582.330000,49564.660000
728,2022-04-05,45883.477500,45702.330000,44512.165000,43913.330000,47491.330000,48090.165000,49280.330000


## Merging Ohlc with Pivot data

In [6]:
mereged_data = merge_ohlc_pivot(resampled_data_1hr,woodie_pivot_1d)

### Generating Buy Signals

In [7]:
traded_buy = generate_buy_pivot_exit(mereged_data,close_col,open_col)

### Generating Sell Signal

In [8]:
traded_sell = generate_sell_pivot_exit(mereged_data,close_col,open_col)

### Calculating Profit and Loss

In [9]:
pnl_buy = get_pnl(traded_buy,close_col)

In [10]:
pnl_sell = get_pnl(traded_sell,close_col)

In [11]:
traded_buy.to_excel(f'woodie_pivot_buy_{file_time}.xlsx', index=False)

In [12]:
traded_sell.to_excel(f'woodie_pivot_sell_{file_time}.xlsx', index=False)

In [13]:
m1 = pd.concat([traded_buy,traded_sell], axis=0)

In [14]:
m1.to_excel(f'./All_trades/woodie_merge_{file_time}.xlsx')

In [15]:
m1 = m1.dropna(subset=['PNL'])

In [16]:
m1.to_excel(f'./report/woodie_merge_{file_time}.xlsx')